Name: Holly Kular\
Date: 03-19-2024\
Email: hkular@ucsd.edu\
decode_L1.m\
Description: wrapper script for decoding analysis on layer 1 of probabilistic RNN

In [15]:
# import these are all needed for fnc
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from scipy.optimize import curve_fit
from scipy.io import loadmat

In [16]:
# add decoding functions
from fnc_fit_and_score import fnc_fit_and_score
from fnc_decode_L1 import fnc_decode_L1


In [33]:
# MODIFY HERE - inputs

# What are we decoding?
RNN_params = {}
RNN_params['prob_split'] = '70_30'
RNN_params['afc'] = 6
RNN_params['coh'] = 'lo'
RNN_params['feedback'] = False
RNN_params['thresh'] = [.3,.7]

# Decoding params
D_params = {}
D_params['time_avg'] = False
D_params['t_win'] = [200,-1]
D_params['n_cvs'] = 5
D_params['num_cgs'] = 30
D_params['label'] = 'stim' # 'stim' or 'choice'
D_params['units'] = 'inh' # 'all' or 'exc' or 'inh'
D_params['pred'] = 'expected' # 'expected' or 'unexpected', trials had stim 0 or 1 dominant

# Timing of task
task_info = {}
task_info['trials'] = 1000
task_info['trial_dur'] = 250  # trial duration (timesteps)
task_info['stim_on'] = 80
task_info['stim_dur'] = 50

# Load data
if sys.platform.startswith('linux'):
    data_dir = f"/mnt/neurocube/local/serenceslab/holly/RNN_Geo/data/rdk_{RNN_params['prob_split']}_{RNN_params['afc']}afc/feedforward_only/{RNN_params['coh']}_coh"
else:
    data_dir = f"/Volumes/serenceslab/holly/RNN_Geo/data/rdk_{RNN_params['prob_split']}_{RNN_params['afc']}afc/feedforward_only/{RNN_params['coh']}_coh"

# Chose the model
mat_files = [f for f in os.listdir(data_dir) if f.endswith('.mat')]# Get all the trained models (should be 40 .mat files)
model_path = os.path.join(data_dir, mat_files[0]) 
model = loadmat(model_path) # model.keys()

# Change this if we want a different number of trials and different default stim (0 is the one RNN was trained on)
if D_params['pred'] == 'expected':
    data_file = f"{data_dir}/Trials{task_info['trials']}_model{model_path[-7:-4]}_0expected.npz"
elif D_params['pred'] == 'unexpected':
    data_file = f"{data_dir}/Trials{task_info['trials']}_model{model_path[-7:-4]}_1unexpected.npz"


In [34]:
# Do decoding
acc = fnc_decode_L1(RNN_params, D_params, data_file, task_info, model)
print(f'done decoding')
# Save to npz
full_file = os.path.join(data_dir, f"decode_T{task_info['trials']}_model{model_path[-7:-4]}_{D_params['label']}_{D_params['pred']}_{D_params['units']}.npz")
np.savez(full_file, decoding_acc = acc)
print(f'done saving')

done decoding
done saving
